##  Import Library

In [1]:
# Import Library
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
from pandas_datareader import data as web
import os
import string
import requests
import nltk
from nltk import word_tokenize
from nltk.corpus import PlaintextCorpusReader

### Fetching Stock Price From Yahoo Finance

In [2]:
# Stock Price
def stock(stock_type, start, end):
    sp500stock = web.get_data_yahoo([stock_type], start,end)
    return sp500stock
    
stock_type = '^GSPC'
start=datetime.datetime(2016, 9, 26)
end=datetime.datetime(2018,10, 1)

stock_data = stock(stock_type, start, end)

In [3]:
stock_data.head(5)

Attributes,High,Low,Open,Close,Volume,Adj Close
Symbols,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,,
2016-09-26,2158.540039,2145.040039,2158.540039,2146.100098,3216170000,2146.100098
2016-09-27,2161.129883,2141.550049,2146.040039,2159.929932,3437770000,2159.929932
2016-09-28,2172.399902,2151.790039,2161.850098,2171.370117,3891460000,2171.370117
2016-09-29,2172.669922,2145.199951,2168.899902,2151.129883,4249220000,2151.129883
2016-09-30,2175.300049,2156.510010,2156.510010,2168.270020,4173340000,2168.270020


#### Due to stock price date is not continuously, so we need to find the date with open markcket

In [4]:
open_date = stock_data.index
len(open_date)

508

### Web Scraping from Wall Street Journal

In [5]:
#Scraping news from the the corresponding date we found
date_loop_list = []
for i in range(len(open_date)):
    d1 = open_date[i]
    date_loop_list.append(d1.date())
print(len(date_loop_list))

508


In [6]:
# Web Scraping from Wall Street Journal
def news(dateloop):
#     date_list = [base - datetime.timedelta(days=x) for x in range(daterange)]
#     date_loop_list = []
#     for i in date_list:
#         dt = datetime.date.strftime(i, '%Y-%m-%d')
#         date_loop_list.append(dt)
    
    All_News=[]
    for date in dateloop:
        url = 'http://www.wsj.com/public/page/archive-'+ str(date) + '.html'
        page = requests.get(url)    
        result = BeautifulSoup(page.content,'lxml')
        UL = result.find_all('ul', class_ = 'newsItem')
        body1 = UL[0].find_all('p')
        everyday_results = []
        Headlines_Day = []
        Abstracts_Day = []
        for news_ in range(len(body1)):
            body2 = body1[news_].get_text()
            body3 = body2.strip().split('\n ')
            Headlines_item = body3[0]
            Abstracts_item = body3[-1]
            Headlines_Day.append(Headlines_item)
            Abstracts_Day.append(Abstracts_item)
        everyday_results = [date, Headlines_Day, Abstracts_Day]
        All_News.append(everyday_results)
    return All_News

#### What news data looks like after scraping from website

In [7]:
output_news = news(date_loop_list)
# output_news

### Convert the web scraping results to dataframe

In [8]:
def news_to_dataframe(output_news):
    news_df = pd.DataFrame()
    date = []
    news_title = []
    news_abstract = []
    for day in output_news:
        date.append(day[0])  # date
        news_title.append(day[1]) # news_title
        news_abstract.append(day[2]) # news_abstract
    news_df['Date'] = date
    news_df['News_title'] = news_title
    news_df['News_abstract'] = news_abstract
    
    return news_df


In [9]:
news_data = news_to_dataframe(output_news) 
news_data.head(10)

,Date,News_title,News_abstract
0,2016-09-26,"[Indian Stocks Emerge From the Crowd , Donald ...",[ Indian shares are on a roll in 201...
1,2016-09-27,"[Takata Air Bags Recalled in South Korea , ペレス...",[ South Korea’s transport ministry h...
2,2016-09-28,"[Photos of the Day:Asia , Missile System That ...",[ A man visits the Forbidden City in...
3,2016-09-29,"[Photos of the Day: Asia , Traders to Scour Ja...",[ Bodybuilders compete over their mu...
4,2016-09-30,[Vietnam to Launch New Benchmark Stock Index ...,[ Vietnam said it will launch VNX Al...
5,2016-10-03,"[U.S. Auto Sales Slipped in September , Nobel ...",[ Light-vehicle sales sputtered in t...
6,2016-10-04,[Derrick Rose’s Trial Begins as Knicks Start P...,[ Knicks guard Derrick Rose’s civil ...
7,2016-10-05,[Oregon Standoff Leader Ammon Bundy Testifies ...,[ The leader of a 41-day standoff at...
8,2016-10-06,[North Korea Activity at Nuclear Sites Raises ...,[ Satellite images showing vehicles ...
9,2016-10-07,[U.S. Carriers to Allow Second Samsung Phone R...,[ U.S. phone carriers said Friday th...


In [10]:
stock_data.to_csv("Stock_Price_kc3214.csv", header = True) 
news_data.to_csv("news_kc3214.csv", header = True)